## 【問題1】BoWとN-gram(手計算)

In [0]:
import collections
import re
import math

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import keras
from keras import optimizers
from keras.models import Model, Sequential, load_model
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D
from keras.layers import Dense, Input, Dropout, Activation, Flatten, LSTM, GRU, Conv1D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [5]:
vocabulary = ["今", "撮影", "中", "です", "休憩", "今日", "ドラマ", "映画", "瞬", "公開"]
ms_kk_texts = ["今撮影中です",
                              "今休憩中です", 
                              "今日ドラマ撮影です", 
                              "今日映画瞬公開です"]
texts_vec = [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], 
                        [1, 0, 1, 1, 1, 0, 0, 0, 0, 0], 
                        [0, 1, 0, 1, 0, 1, 1, 0, 0, 0],
                        [0, 0, 0, 1, 0, 1, 0, 1, 1, 1]]

df_bow_1gram = pd.DataFrame(data = texts_vec, columns = vocabulary, index = ms_kk_texts)
df_bow_1gram

,今,撮影,中,です,休憩,今日,ドラマ,映画,瞬,公開
今撮影中です,1,1,1,1,0,0,0,0,0,0
今休憩中です,1,0,1,1,1,0,0,0,0,0
今日ドラマ撮影です,0,1,0,1,0,1,1,0,0,0
今日映画瞬公開です,0,0,0,1,0,1,0,1,1,1


## 【問題2】TF-IDF(手計算)

In [0]:
def calc_term_frequency(word, text, vec):
    word_frequency_count = text.count(word)
    n_word_in_text = sum(vec)

    return word_frequency_count / n_word_in_text


def calc_inverse_document_frequency(word, texts, vecs, w_index):
    text_count = len(texts)
    text_count_included_word = sum([vec[w_index] for vec in vecs])

    return math.log2(text_count / text_count_included_word)


def calc_tf_idf(words, texts, vecs, w_index, t_index):  
    tf = calc_term_frequency(words[w_index], texts[t_index], vecs[t_index])
    idf = calc_inverse_document_frequency(words[w_index], texts, vecs, w_index)
                                  
    return tf * idf

In [0]:
# "今"のインデックスを取得
w_index = vocabulary.index("今")
# "今撮影中です"のインデックスを取得
t_index = ms_kk_texts.index("今撮影中です")

print(calc_tf_idf(vocabulary, ms_kk_texts, texts_vec, w_index, t_index))

# "です"のインデックスを取得
w_index = vocabulary.index("です")
# "今日ドラマ撮影です"のインデックスを取得
t_index = ms_kk_texts.index("今日ドラマ撮影です")

print(calc_tf_idf(vocabulary, ms_kk_texts, texts_vec, w_index, t_index))

# "瞬"のインデックスを取得
w_index = vocabulary.index("瞬")
# "今日映画瞬公開です	"のインデックスを取得
t_index = ms_kk_texts.index("今日映画瞬公開です")

print(calc_tf_idf(vocabulary, ms_kk_texts, texts_vec, w_index, t_index))

0.25
0.0
0.4


## 【問題3】テキストクリーニング(プログラミング)

In [0]:
# 対象テキストデータ
text = "<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:"

# 正規表現一覧
symbol_reg = r'[\n*` ]+'
mention_reg = r'<.*?>'
phraze_reg = r'【.*?】'
command_reg = r':[^0-9０-９]+:'

reg_str = f'{symbol_reg}|{mention_reg}|{phraze_reg}|{command_reg}'
reg_str = re.compile(reg_str)
hoge = re.sub(reg_str, '', text)
# re.sub(r'[\n*！`]+', '', text)でもできます
print(hoge)

有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！


## 【問題4】形態素解析
janomeをcolab上にインストール・インポート  
使い方：https://note.nkmk.me/python-janome-tutorial/

In [0]:
!pip install janome

     |████████████████████████████████| 25.1MB 2.8MB/s 


In [0]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
for token in t.tokenize(hoge):
    print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
6	名詞,数,*,*,*,*,6,*,*
ヶ月	名詞,接尾,助数詞,*,*,*,ヶ月,カゲツ,カゲツ
間	名詞,接尾,一般,*,*,*,間,カン,カン
100	名詞,数,*,*,*,*,100,*,*
%	名詞,サ変接続,*,*,*,*,%,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
ご	接頭詞,名詞接続,*,*,*,*,ご,ゴ,ゴ
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
！	記号,一般,*,*,*,*,！,！,！
この	連体詞,*,*,*,*,*,この,コノ,コノ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
、	記号,読点,*,*,*,*,、,、,、
RubyMine	名詞,固有名詞,組織,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
で	助

In [0]:
print([token.surface for token in t.tokenize(hoge)
       if token.part_of_speech.split(',')[0] in ['名詞', '動詞']])

['有償', 'RubyMine', 'PyCharm', '6', 'ヶ月', '間', '100', '%', 'OFF', 'クーポン', '希望', '者', '方', '先着', '100', '名', '様', '贈呈', 'いたし', '度', 'RubyMine', 'PyCharm', 'メーカー', 'JetBrains', '社', 'クーポン', 'コード', '提供', '交渉', '実り', '100', 'クーポン', 'いただく', 'こと', 'でき', 'RubyMinePyCharm', '希望', '方', '手', '挙げ', '方式', '希望', '方', '投稿', '手', 'あげ', 'スタンプ', 'クリック', 'し', 'ください', '期限', '2019', '年', '3', '月', '20', '日', '水', '22', ':', '00', 'さ', 'せ', 'いただき', '希望', 'お待ち', 'し', 'おり']


## 【問題5】ニュースの分析

In [0]:
# livedoorのnewsをダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# 圧縮ファイルを解凍
!tar zxf ldcc-20140209.tar.gz
# livedoorニュースの説明を表示
#!cat text/README.txt

--2019-07-01 01:36:16--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.68MB/s    in 3.6s    

2019-07-01 01:36:21 (2.35 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



 - まずどんなニュースなのか読んでみる

In [0]:
# encodingをutf-8指定して読み込み
bin_data = load_files('./text', encoding='utf-8')
documents = bin_data.data
# 今回はラベルが無いと仮定してください
# targets = bin_data.target
documents[0]

'http://news.livedoor.com/article/detail/4931238/\n2010-08-08T10:00:00+0900\nNY名物イベントが日本でも！名店グルメを気軽に楽しむ\nニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\n\u3000\n\u3000実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。\n\n参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。\n\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\n\n\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？\n\nJAPAN RESTAURANT WEEK 2010 -公式サイト\n'

- 出現単語をカウントして分析する

In [0]:
t = Tokenizer()
c = collections.Counter(t.tokenize(documents[0], wakati=True))
print(c)

Counter({'、': 28, 'の': 17, 'に': 14, 'を': 11, ' ': 11, '「': 8, '・': 8, '」': 8, 'が': 7, 'て': 7, 'レストラン': 7, '。': 7, 'は': 7, '\n': 6, 'で': 6, '2010': 5, '/': 4, 'イベント': 4, '日本': 4, '\u3000': 4, '-': 3, '！': 3, 'ウィーク': 3, '版': 3, '特別': 3, '月': 3, '日': 3, '期間': 3, 'し': 3, 'です': 3, '服部': 3, '氏': 3, '田村': 3, '\n\n': 3, '参加': 3, '.': 2, '08': 2, ':': 2, '00': 2, 'も': 2, '気軽': 2, 'いる': 2, '食': 2, '協賛': 2, '8': 2, 'まで': 2, '店舗': 2, 'た': 2, '円': 2, '（': 2, '税': 2, 'サ': 2, '別': 2, '）': 2, '楽しめる': 2, '実行': 2, '委員': 2, '長': 2, '学校': 2, 'ある': 2, 'さん': 2, 'シェフ': 2, 'つき': 2, 'ぢ': 2, 'など': 2, '料理': 2, 'と': 2, 'できる': 2, '予約': 2, 'ジャパン': 2, 'http': 1, '://': 1, 'news': 1, 'livedoor': 1, 'com': 1, 'article': 1, 'detail': 1, '4931238': 1, 'T': 1, '10': 1, '+': 1, '0900': 1, 'NY': 1, '名物': 1, '名店': 1, 'グルメ': 1, '楽しむ': 1, 'ニューヨーク': 1, '20': 1, '年': 1, '続い': 1, '祭典': 1, 'その': 1, 'ダイナーズクラブ': 1, 'もと': 1, '7': 1, '30': 1, 'より': 1, 'スタート': 1, '31': 1, '中': 1, '青山': 1, '六本木': 1, '丸の内': 1, '銀座': 1, '横浜': 1, 'エリア': 1

単語で一番多かったのは「レストラン」、ついで「イベント」「日本」  
助詞と「、」が多かった。記号が多いのでクリーニングしたほうがよさそう。  


<br>

- テキストをクリーニングする

In [0]:
# 正規表現一覧
symbol_reg = r'[\n*`\-\+\[\]\'…\/「」・（）■“”『』／()−＜＞［］【】◯◆\- 　]+'
time_reg = r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{4}'
url_reg = r'https?:\/\/[\w\/:%#\$&\?\(\)~\.=\+\-]+'

reg_str = f'{symbol_reg}|{time_reg}|{url_reg}'
reg_str = re.compile(reg_str)

cleaned_docs = []

# クリーニング
for doc in documents:
    cleaned_docs.append(re.sub(reg_str, '', doc))

print(cleaned_docs[:10])

['NY名物イベントが日本でも！名店グルメを気軽に楽しむニューヨークで20年続いている食の祭典レストランウィーク。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円税サ別、ディナー5000円税サ別を気軽に楽しめる、とっておきのイベントです。実行委員長には、学校法人服部学園、服部栄養専門学校理事長校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LABETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフのレストランタテルヨシノ汐留や、日本料理の名門つきぢ田村、金田中庵、赤坂璃宮にmikuniMARUNOUCHIなど、日本を代表するレストランがずらり。イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。予約方法は必ず事前に、各店舗に問合せを行いジャパンレストランウィーク2010での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できるジャパンレストランウィーク2010にぜひ参加しててみてはいかがですか？JAPANRESTAURANTWEEK2010公式サイト', '小沢一郎氏の妻が支援者に離婚を報告。週刊文春報じる13日、Web版週刊文春は、民主党の元代表小沢一郎氏の妻が、支援者宛に離婚したことを伝える手紙を送ったと報じ、ツイッターやネット掲示板で大きな話題になっている。記事によると、その手紙は小沢は放射能が怖くて秘書と一緒に逃げだしました隠し子が発覚した際、小沢元代表は和子夫人に謝るどころか、いつでも離婚してやると言い放ち、和子夫人は一時は自殺まで考えたという小沢氏の支持者にとってはショッキングな内容となっている。ツイッタ

- BoW + TFIDFでベクトル化する


In [0]:
# 200記事をチョイス
choosed_docs = cleaned_docs[:500]

In [0]:
def get_token(text):
    t = Tokenizer()
    tokens = t.tokenize(text)
    word = ""
    for token in tokens:
        part_of_speech = token.part_of_speech.split(",")[0]
        if part_of_speech == "名詞":
            word +=token.surface + " "
        if part_of_speech == "動詞":
            word +=token.base_form+ " "
        if part_of_speech == "形容詞":
            word +=token.base_form+ " "
        if part_of_speech == "形容動詞":
            word +=token.base_form+ " "
    return word

In [0]:
tokens = np.array([])

for doc in choosed_docs:
    token = get_token(doc)
    tokens = np.append(tokens, token)

In [0]:
c_vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
tf = c_vectorizer.fit_transform(tokens)

t_transformer = TfidfTransformer()
tfidf = t_transformer.fit_transform(tf).toarray()

In [0]:
def calc_cosine_similarity(v1, v2):
    """
    ベクトルv1, v2のcos類似度の算出
    """
    return sum([a*b for a, b in zip(v1, v2)])/(sum(map(lambda x: x*x, v1))**0.5 * sum(map(lambda x: x*x, v2))**0.5)

- あるニュースに一番cos類似度が近いニュースを出力する関数の作成
- 別の類似度手法を1つ調べて関数に組み込む(切り替えられるようにする)


In [0]:
# 最初の記事
v1 = tfidf[0]
sims = np.array([])

for index in range(len(tfidf)):
    v2 = tfidf[index]
    
    if np.allclose(v1, v2) == False:
        sims = np.append(sims, calc_cosine_similarity(v1, v2))

highest_similarity = np.argmax(sims)
print(sims[highest_similarity])
print(highest_similarity)

0.0923204502645693
271


In [0]:
print(choosed_docs[0])
print(tokens[0])
print(choosed_docs[highest_similarity])
print(tokens[highest_similarity])
print(re.findall(url_reg, documents[0]))
print(re.findall(url_reg, documents[highest_similarity]))

NY名物イベントが日本でも！名店グルメを気軽に楽しむニューヨークで20年続いている食の祭典レストランウィーク。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円税サ別、ディナー5000円税サ別を気軽に楽しめる、とっておきのイベントです。実行委員長には、学校法人服部学園、服部栄養専門学校理事長校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LABETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフのレストランタテルヨシノ汐留や、日本料理の名門つきぢ田村、金田中庵、赤坂璃宮にmikuniMARUNOUCHIなど、日本を代表するレストランがずらり。イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。予約方法は必ず事前に、各店舗に問合せを行いジャパンレストランウィーク2010での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できるジャパンレストランウィーク2010にぜひ参加しててみてはいかがですか？JAPANRESTAURANTWEEK2010公式サイト
NY 名物 イベント 日本 名店 グルメ 気軽 楽しむ ニューヨーク 20 年 続く いる 食 祭典 レストラン ウィーク 日本 版 ダイナーズクラブ 特別 協賛 もと 7 月 30 日 スタート 8 月 31 日 期間 中 青山 六本木 丸の内 銀座 横浜 エリア ラグジュアリーレストラン 81 店舗 イベント ため 特別 用意 する ランチ メニュー 2010 円 税 サ 別 ディナー 5000 円 税 サ 別 気軽 楽しめる とる おく イベント 実行 委員 長 学校

- なぜそのような結果になったのか考察する  
記事の内容的には全く類似性は見れれなかったため、名詞などよりも「する」「いる」などの動詞の頻出具合で類似度が高いと判断された？

## 【問題6】感情分析
https://www.tensorflow.org/tutorials/keras/basic_text_classification

In [0]:
# IMDBをカレントフォルダにダウンロード
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# 解凍
!tar zxf aclImdb_v1.tar.gz
# aclImdb/train/unsupはラベル無しのため削除
!rm -rf aclImdb/train/unsup
# IMDBデータセットの説明を表示
#!cat aclImdb/README

In [0]:
train_review = load_files('./aclImdb/train/', encoding='utf-8')
train_text, train_y = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_text, test_y = test_review.data, test_review.target

In [6]:
train_text[:10]

["Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.",
 'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clich

前処理は下記を実施  
- 記号・htmlタグ・lfの削除
- lemmatize
- 小文字で統一
- text → sequenceに変換
- 最大長のsequenceに併せてパディング

In [0]:
def clean_text(text):
    text = re.sub(reg_str,'',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [8]:
max_features = 6000

# 正規表現一覧
symbol_reg = r'[^\w\s]'
html_reg = r'<.*?>'
lf_reg = r'\\\'[a-z]'
reg_str = f'{symbol_reg}|{html_reg}|{lf_reg}'
reg_str = re.compile(reg_str)

total_reviews = train_text + test_text


nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()


for index in range(len(total_reviews)):
    total_reviews[index] = clean_text(total_reviews[index])

    
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(total_reviews)


for index in range(len(train_text)):
    train_text[index] = clean_text(train_text[index])

    
for index in range(len(test_text)):
    test_text[index] = clean_text(test_text[index])


train_tokens = tokenizer.texts_to_sequences(train_text)
test_tokens = tokenizer.texts_to_sequences(test_text)

# pad sequences
max_length = max([len(s.split()) for s in total_reviews])
train_x = pad_sequences(train_tokens, maxlen=max_length, padding='post')
test_x = pad_sequences(test_tokens, maxlen=max_length, padding='post')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
# set parameters:
batch_size = 100
embedding_dims = 128
epochs = 100


print('Build model...')

model = Sequential()
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=max_length))
model.add(LSTM(64, return_sequences = True))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


print('Train...')

es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
cp_cb = ModelCheckpoint(filepath = 'imdb_{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_data=(test_x, test_y), callbacks=[es_cb,cp_cb, learning_rate_reduction])

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/100
25000/25000 [==============================] - 969s 39ms/step - loss: 0.4779 - acc: 0.7725 - val_loss: 0.3250 - val_acc: 0.8612

Epoch 00001: val_loss improved from inf to 0.32503, saving model to imdb_01-0.33.hdf5
Epoch 2/100
25000/25000 [==============================] - 966s 39ms/step - loss: 0.2561 - acc: 0.9014 - val_loss: 0.3029 - val_acc: 0.8717

Epoch 00002: val_loss improved from 0.32503 to 0.30285, saving model to imdb_02-0.30.hdf5
Epoch 3/100
25000/25000 [==============================] - 968s 39ms/step - loss: 0.1925 - acc: 0.9299 - val_loss: 0.3283 - val_acc: 0.8690

Epoch 00003: val_loss did not improve from 0.30285
Epoch 4/100
25000/25000 [==============================] - 963s 39ms/step - loss: 0.1511 - acc: 0.9455 - val_loss: 0.3411 - val_acc: 0.8633

Epoch 00004: val_loss did not improve from 0.30285
Epoch 00004: early stopping


90%超えない、、  
モデルを変えて試してみる

In [31]:
# set parameters::
batch_size = 100
embedding_dims = 128
epochs = 100
layers = 2
learning_rate=1e-3
units=64
dropout_rate=0.2

print('Build model...')

model = Sequential()
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=max_length))
model.add(Flatten())

for _ in range(layers-1):
    model.add(Dense(units=units, activation='relu'))
    model.add(Dropout(rate=dropout_rate))

model.add(Dense(units=1, activation='sigmoid'))


optimizer = optimizers.Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

print('Train...')

es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
cp_cb = ModelCheckpoint(filepath = 'imdb_{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_data=(test_x, test_y), verbose=2, callbacks=[es_cb,cp_cb, learning_rate_reduction])

Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/100
 - 12s - loss: 0.5124 - acc: 0.7640 - val_loss: 0.3135 - val_acc: 0.8680

Epoch 00001: val_loss improved from inf to 0.31349, saving model to imdb_01-0.31.hdf5
Epoch 2/100
 - 11s - loss: 0.2206 - acc: 0.9179 - val_loss: 0.3214 - val_acc: 0.8638

Epoch 00002: val_loss did not improve from 0.31349
Epoch 3/100
 - 11s - loss: 0.0794 - acc: 0.9786 - val_loss: 0.3728 - val_acc: 0.8614

Epoch 00003: val_loss did not improve from 0.31349
Epoch 00003: early stopping


あまり変わらない、、  
次は前処理をtfidfに変えてみる

In [0]:
train_review = load_files('./aclImdb/train/', encoding='utf-8')
train_text, train_y = train_review.data, train_review.target

test_review = load_files('./aclImdb/test/', encoding='utf-8')
test_text, test_y = test_review.data, test_review.target

In [0]:
def ngram_vectorize(train_texts, train_labels, val_texts):
    NGRAM_RANGE = (1, 2)
    TOP_K = 20000
    TOKEN_MODE = 'word'
    MIN_DOC_FREQ = 2
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ,
    }
    
    # Learn Vocab from train texts and vectorize train and val sets
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    x_train = tfidf_vectorizer.fit_transform(train_texts)
    x_val = tfidf_vectorizer.transform(val_texts)
    
    # Select best k features, with feature importance measured by f_classif
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

In [21]:
layers = 2
learning_rate=1e-3
epochs=100
batch_size=128
units=64
dropout_rate=0.2


# Vectorize the data
x_train, x_val = ngram_vectorize(train_text, train_y, test_text)

print('Build model...')
model = Sequential()
model.add(Dropout(rate=dropout_rate, input_shape=x_train.shape[1:]))

for _ in range(layers-1):
    model.add(Dense(units=units, activation='relu'))
    model.add(Dropout(rate=dropout_rate))

model.add(Dense(units=1, activation='sigmoid'))


optimizer = optimizers.Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

# Create callback for early stopping on validation loss. If the loss does
# not decrease on two consecutive tries, stop training
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]


print('Train...')
# Train and validate model
history = model.fit(x_train, train_y, epochs=epochs, validation_data=(x_val, test_y),
                    verbose=2, batch_size=batch_size, callbacks=callbacks)

# Print results
history = history.history
val_acc = history['val_acc'][-1]
val_loss = history['val_loss'][-1]
print ('Validation accuracy: {acc}, loss: {loss}'.format(
        acc=val_acc, loss=val_loss))

# Save model
model.save('IMDB_mlp_model_' + str(val_acc) + '_binary_crossentropy'+ '.h5')

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


Train on 25000 samples, validate on 25000 samples
Epoch 1/100
 - 4s - loss: 0.4731 - acc: 0.8601 - val_loss: 0.3263 - val_acc: 0.8864
Epoch 2/100
 - 3s - loss: 0.2348 - acc: 0.9208 - val_loss: 0.2519 - val_acc: 0.9016
Epoch 3/100
 - 3s - loss: 0.1692 - acc: 0.9406 - val_loss: 0.2337 - val_acc: 0.9061
Epoch 4/100
 - 3s - loss: 0.1351 - acc: 0.9529 - val_loss: 0.2300 - val_acc: 0.9050
Epoch 5/100
 - 3s - loss: 0.1103 - acc: 0.9632 - val_loss: 0.2332 - val_acc: 0.9041
Epoch 6/100
 - 3s - loss: 0.0953 - acc: 0.9686 - val_loss: 0.2404 - val_acc: 0.9020
Validation accuracy: 0.90196, loss: 0.240361744556427


90を超えた...  
lstmに変えればさらに気持ち上がるかも？





## 【問題8】自然言語処理の応用事例
http://www.textmining.jp/case_keyword.html  
http://www.textmining.jp/game_platform.html